In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson
from iminuit import Minuit
from iminuit.cost import LeastSquares
import math
import matplotlib.patches as patches

In [ ]:
#resistenza, capacità e frequenza di taglio attesa
R = 
C = 
Fexp = 1/(R*C*2*np.pi)
print("la frequenza di taglio attesa è {} Hz".format(Fexp))

In [ ]:
#Dati resistore

#frequenze
Fr = np.array([])

#V in entrata
errVinr = np.array([])
Vinpr = np.array([])
lr = len(Vinpr)
Vinr = np.full(len(Vinpr), Vinpr.mean())

#V in uscita
errVoutr = np.array([])
Voutr = np.array([])

#Attenuazione
Ar = Voutr/Vinr
errAr = Ar*np.sqrt((errVinr/Vinr)**2+(errVoutr/Voutr)**2)

#Fase
phigradr = np.array([])
errphigradr = np.array([1 for i in range(len(phigradr))])
phir = (phigradr/180)*np.pi
errphir = (errphigradr/180)*np.pi

In [ ]:
#Dati capacitore

#frequenze
Fc = np.array([])

#V in entrata
errVinc = np.array([])
Vinpc = np.array([])
lc = len(Vinpc)
Vinc = np.full(len(Vinpc), Vinpc.mean())

#V in uscita
errVoutc = np.array([])
Voutc = np.array([])

#Attenuazione
Ac = Voutc/Vinc
errAc = Ac*np.sqrt((errVinc/Vinc)**2+(errVoutc/Voutc)**2)

#Fase
phigradc = np.array([])
errphigradc = np.array([1 for i in range(len(phigradc))])
phic = (phigradc/180)*np.pi
errphic = (errphigradc/180)*np.pi

In [ ]:
#Grafico
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

#Attenuazione
axes[0].errorbar(Fr, Ar, yerr = errAr, label='Condensatore', color='blue', fmt = 'o')
axes[0].errorbar(Fc, Ac, yerr = errAc, label='Resistore', color='red', fmt = 'o')
axes[0].set_title('Attenuazione')
axes[0].legend()
axes[0].set_xscale('log')
axes[0].grid(True)
axes[0].set_xlabel("Frequenza (Hz)")
axes[0].set_ylabel("Attenuazione")
axes[0].axhline(y=1/np.sqrt(2), color='red', linestyle='--', linewidth=1)

#Rettangolo per evidenziare i dati di fit
rect = patches.Rectangle(
    (9000, 0.55),          # angolo inferiore sinistro (x, y)
    20000 - 9000,          # larghezza
    0.8 - 0.5,             # altezza
    linewidth=2,
    edgecolor='red',
    facecolor='yellow',
    alpha=0.3              # trasparenza
)
axes[0].add_patch(rect)

#Fase
axes[1].errorbar(Fr, phir, yerr = errphir, label='Resistore', color='blue', fmt = 'o')
axes[1].errorbar(Fc, phic, yerr = errphic, label='Condensatore', color='red', fmt = 'o')
axes[1].set_title('Fase')
axes[1].legend()
axes[1].set_xscale('log')
axes[1].grid(True)
axes[1].set_xlabel("Frequenza (Hz)")
axes[1].set_ylabel("Fase (rad)")


In [ ]:
#Fit attenuazione resistore

def ret(x,a,b):
    return(a*x+b)
least_squares_r = LeastSquares(Fr[:], Ar[:], errAr[:], ret)
mr = Minuit(least_squares_r, a = , b =  )
mr.migrad()
mr.minos()


In [ ]:
#Fit condensatore

least_squares_c = LeastSquares(Fc[:], Ac[:], errAc[:], ret)
mc = Minuit(least_squares_c, a = , b =  )
mc.migrad()
mc.minos()

In [ ]:
#Grafico per l'intersezione delle rette

fi, ax = plt.subplots(figsize=(10, 4))
p = (min(Fc[:].min(),Fr[:].min())*0.9,max(Fc[:].max(),Fr[:].max())*1.1)

#attenuazione
ax.errorbar(Fc[:], Ac[:], yerr=errAc[:], label='Condensatore', color='blue', fmt = 'o' )
ax.errorbar(Fr[:], Ar[:], yerr=errAr[:], label='Resistore', color='red', fmt = 'o')
ax.set_title('Attenuazione')
ax.legend()
ax.grid(True)
ax.plot(p,ret(p, mr.values['a'], mr.values['b']), color = 'lightcoral', label = 'fit resistore')
ax.plot(p,ret(p, mc.values['a'], mc.values['b']), color = 'skyblue', label = 'fit condensatore')
ax.set_xlabel("Frequenza (Hz)")
ax.set_ylabel("Attenuazione")

In [ ]:
#Risultati
vt = (mr.values['b']-mc.values['b'])/(mc.values['a']-mr.values['a'])

e_a_res = mr.errors['a']
e_a_cond = mc.errors['a']
e_b_res = mr.errors['b']
e_b_cond = mc.errors['b']
cov_res = mr.covariance[('a', 'b')]
cov_cond = mc.covariance[('a', 'b')]


delfina = (((mr.values['b']-mc.values['b'])/((mc.values['a']-mr.values['a'])**2))**2)*(e_a_res**2+e_a_cond**2)
diego = ((1/(mc.values['a']-mr.values['a']))**2)*(e_b_res**2+e_b_cond**2)
claudio = 2*((mr.values['b']-mc.values['b'])/((mc.values['a']-mr.values['a'])**3))*(cov_res+cov_cond)

errvt = (delfina + diego + claudio)**0.5
print("La frequenza stimata è {} +/- {} Hz".format(vt, errvt))

#test di consistenza

test = abs((vt - Fexp)/(errvt)) 
print("Consistente con Fexp entro {} sigma".format(test))